# 데이터의 특징
해당 데이터는 인공적으로 만들어졌습니다.
또한 각 피처와 타깃값의 의미를 알 수 없습니다. 순전히 데이터만 보고 접근해야 합니다.

### TARGET = 0 or 1

- bin_ = 이진 피처
- nom_ = 명목형 피처
- ord_ = 순서형 피처
    - Ex. ord_3, ord_4, ord_5와 같이 알파벳순으로 고유값 순서를 매겼다고 한다.
- day, month = 날짜 피처

In [ ]:
import pandas as pd

data_path = '../input/cat-in-the-dat/'

# index column 추가 ('id')
train = pd.read_csv(data_path + 'train.csv', index_col='id')
test = pd.read_csv(data_path + 'test.csv', index_col='id')
submission = pd.read_csv(data_path + 'sample_submission.csv', index_col='id')

train.shape, test.shape

In [ ]:
train. head().T

### 제출 샘플 데이터 
- target 값은 0.5로 되어있습니다.
- 즉 테스트 데이터의 target 값이 1일 확률을 예측합니다.

In [ ]:
submission.head()

### Feature 요약표
고유값, 결측값, 실제 입력값 등을 정리한 표

In [ ]:
def resumetable(df):
    print(f'데이터셋 형상: {df.shape}')
    summary = pd.DataFrame (df.dtypes, columns=['데이터 타입'] ) 
    summary = summary.reset_index()
    summary = summary.rename(columns={'index' : '피처'} )
    summary['결측값 개수'] = df.isnull().sum().values
    summary['고윷값 개수'] = df.nunique().values
    summary['첫 번째 값'] =df.loc[0].values
    summary['두 번째 값'] = df.loc[1].values
    summary['세 번째 값'] = df.loc[2].values
    return summary
resumetable(train)

T = True, Y = Yes -> 1         
F = False, N = No -> 0          

ord 타입은 순서형 데이터입니다. 순서가 중요합니다.       
ord 피처의 고유값을 출력하겠습니다.       

unique 함수를 통해 고유값의 등장순으로 출력할 수 있습니다.

ord_0은 그저 숫자로 보입니다.      
ord_1은 캐글의 등급입니다. 등급에 따라 Novice, Contributor, Expert, Master, Grandmaster 순으로 맞추어주면 되겠습니다.        
ord_2은 춥고 더운 정도입니다.

In [ ]:
for i in range(3):
    feature = 'ord_' + str(i)
    print(f'{feature} 고유값 : {train[feature].unique()}')

### ord_3 ~ ord_5 피처
- 알파벳 순으로 정렬

In [ ]:
for i in range(3, 6):
    feature = 'ord_' + str(i)
    print(f'{feature} 고유값 : {train[feature].unique()}')

### dat, month의 고유값

In [ ]:
print('day 고웃값 : ', train['day'].unique())
print('month 고웃값 : ',train['month'].unique())

## Data 시각화

In [ ]:
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
def write_percent(ax, total_size):
    '''도형 객체를 순회하며 막대 상단에 타깃값 비율 표시'''
    for patch in ax.patches:
        height = patch.get_height()     # 도형 높이(데이터 개수)
        width = patch.get_width()       # 도형 너비
        left_coord = patch.get_x()      # 도형 왼쪽 테두리의 x축 위치
        percent = height/total_size*100 # 타깃값 비율
        
        # (x, y) 좌표에 텍스트 입력 
        ax.text(x=left_coord + width/2.0,    # x축 위치
                y=height + total_size*0.001, # y축 위치
                s=f'{percent:1.1f}%',        # 입력 텍스트
                ha='center')                 # 가운데 정렬

plt.figure(figsize=(7, 6))

ax = sns.countplot(x='target', data=train)
write_percent(ax, len(train)) # 비율 표시
ax.set_title('Target Distribution');

### 이진 피처 분포

In [ ]:
import matplotlib.gridspec as gridspec # 여러 그래프를 격자 형태로 배치

# 3행 2열 틀(Figure) 준비
mpl.rc('font', size=12)
grid = gridspec.GridSpec(3, 2) # 그래프(서브플롯)를 3행 2열로 배치
plt.figure(figsize=(10, 16))   # 전체 Figure 크기 설정
plt.subplots_adjust(wspace=0.4, hspace=0.3) # 서브플롯 간 좌우/상하 여백 설정

# 서브플롯 그리기
bin_features = ['bin_0', 'bin_1', 'bin_2', 'bin_3', 'bin_4'] # 피처 목록

for idx, feature in enumerate(bin_features): 
    ax = plt.subplot(grid[idx]) 
    
    # ax축에 타깃값 분포 카운트플롯 그리기
    sns.countplot(x=feature,
                  data=train,
                  hue='target',
                  palette='pastel', # 그래프 색상 설정
                  ax=ax)
    
    ax.set_title(f'{feature} Distribution by Target') # 그래프 제목 설정
    write_percent(ax, len(train))    

### 날짜형 데이터

In [ ]:
# Crosstab
def get_crosstab(df, feature):
    crosstab = pd.crosstab(df[feature], df['target'], normalize='index')*100
    crosstab = crosstab.reset_index()
    return crosstab

In [ ]:

def plot_pointplot(ax, feature, crosstab):
    ax2 = ax.twinx() # x축은 공유하고 y축은 공유하지 않는 새로운 축 생성
    # 새로운 축에 포인트플롯 그리기
    ax2 = sns.pointplot(x=feature, y=1, data=crosstab,
                        order=crosstab[feature].values, # 포인트플롯 순서
                        color='black',                  # 포인트플롯 색상
                        legend=False)                   # 범례 미표시
    ax2.set_ylim(crosstab[1].min()-5, crosstab[1].max()*1.1) # y축 범위 설정
    ax2.set_ylabel('Target 1 Ratio(%)')

In [ ]:
def plot_cat_dist_with_true_ratio(df, features, num_rows, num_cols, 
                                  size=(15, 20)):
    plt.figure(figsize=size)  # 전체 Figure 크기 설정
    grid = gridspec.GridSpec(num_rows, num_cols) # 서브플롯 배치
    plt.subplots_adjust(wspace=0.45, hspace=0.3) # 서브플롯 좌우/상하 여백 설정
    
    for idx, feature in enumerate(features): 
        ax = plt.subplot(grid[idx])
        crosstab = get_crosstab(df, feature) # 교차분석표 생성

        # ax축에 타깃값 분포 카운트플롯 그리기
        sns.countplot(x=feature, data=df,
                      order=crosstab[feature].values,
                      color='skyblue',
                      ax=ax)

        write_percent(ax, len(df)) # 비율 표시
       
        plot_pointplot(ax, feature, crosstab) # 포인트플롯 그리기
        
        ax.set_title(f'{feature} Distribution') # 그래프 제목 설정

In [ ]:
date_features = ['day', 'month']
plot_cat_dist_with_true_ratio(train, date_features, 
                              num_rows=2, num_cols=1, size=(10, 10))

### 데이터 합치기
train + test 진행 후 Split       
Concat 함수 사용 : 데이터 프레임 형태가 동일하다면 사용 가능

In [ ]:
all_data = pd.concat([train, test]) # 훈련 데이터와 테스트 데이터 합치기 
all_data = all_data.drop('target', axis=1) # 타깃값 제거
all_data.head(3)

## 원핫 인코더 생성 및 적용 ( 모든 데이터에 대해 )
Baseline이기 때문에 일단 모든 데이터에 원핫 인코딩 적용

In [ ]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder() # 원-핫 인코더 생성
all_data_encoded = encoder.fit_transform(all_data) # 원-핫 인코딩 적용

### Data Split

In [ ]:
num_train = len(train) # 훈련 데이터 개수

# 훈련 데이터와 테스트 데이터 나누기
X_train = all_data_encoded[:num_train] # 0 ~ num_train - 1행
X_test = all_data_encoded[num_train:] # num_train ~ 마지막 행

y = train['target']

from sklearn.model_selection import train_test_split

# 훈련 데이터, 검증 데이터 분리
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y,
                                                      test_size=0.1,
                                                      stratify=y,
                                                      random_state=10)

### Base Model : LogisticRegression

In [ ]:
from sklearn.linear_model import LogisticRegression

logistic_model = LogisticRegression(max_iter=1000, random_state=42) # 모델 생성
logistic_model.fit(X_train, y_train) # 모델 훈련

### Predict

In [ ]:
logistic_model.predict_proba(X_valid)
logistic_model.predict(X_valid)

# 검증 데이터를 활용한 타깃 예측 
y_valid_preds = logistic_model.predict_proba(X_valid)[:, 1]

In [ ]:
from sklearn.metrics import roc_auc_score # ROC AUC 점수 계산 함수

# 검증 데이터 ROC AUC
roc_auc = roc_auc_score(y_valid, y_valid_preds)

print(f'검증 데이터 ROC AUC : {roc_auc:.4f}')

In [ ]:
# 타깃값 1일 확률 예측
y_preds = logistic_model.predict_proba(X_test)[:, 1]

In [ ]:
# 제출 파일 생성
submission['target'] = y_preds
submission.to_csv('submission.csv')

## 성능 개선하기 ~
### 각 피처에 맞게 인코딩 하기
- 이진 피처 인코딩 ( Y, T -> 1 | N, F -> 0 )
- 순서형 피처 -> 1, 2, 3, 4, 5....
- 명목형 피처, 날짜형 피처 -> 원핫 인코딩

### 피처 스케일링
- 순서형 피처 ord_0~5에 대해 min_max 정규화

데이터 다시 로드 및 합치기

In [ ]:
data_path = '../input/cat-in-the-dat/'

# index column 추가 ('id')
train = pd.read_csv(data_path + 'train.csv', index_col='id')
test = pd.read_csv(data_path + 'test.csv', index_col='id')
submission = pd.read_csv(data_path + 'sample_submission.csv', index_col='id')

# 훈련 데이터와 테스트 데이터 합치기
all_data = pd.concat([train, test])
all_data = all_data.drop('target', axis=1) # 타깃 값 제거

In [ ]:
# 이진 피처 인코딩
all_data['bin_3'] = all_data['bin_3'].map({'F':0, 'T':1})
all_data['bin_4'] = all_data['bin_4'].map({'N':0, 'Y':1})

# 순서형 피처 인코딩
# 1. 데이터양이 적은 피처는 Dictionary를 통해 인코딩
ord1dict = {'Novice':0, 'Contributor':1, 
            'Expert':2, 'Master':3, 'Grandmaster':4}
ord2dict = {'Freezing':0, 'Cold':1, 'Warm':2, 
            'Hot':3, 'Boiling Hot':4, 'Lava Hot':5}

all_data['ord_1'] = all_data['ord_1'].map(ord1dict)
all_data['ord_2'] = all_data['ord_2'].map(ord2dict)

In [ ]:
# 2. 데이텅양이 많으면, OrdinalEncoder를 이용하여 인코딩
from sklearn.preprocessing import OrdinalEncoder

ord_345 = ['ord_3', 'ord_4', 'ord_5']

ord_encoder = OrdinalEncoder() # OrdinalEncoder 객체 생성
# ordinal 인코딩 적용
all_data[ord_345] = ord_encoder.fit_transform(all_data[ord_345])

# 피처별 인코딩 순서 출력
for feature, categories in zip(ord_345, ord_encoder.categories_):
    print(feature)
    print(categories)
    
all_data[ord_345]

### 명목형 피처 인코딩

In [ ]:
# nom_0~9까지의 List[str] 생성
nom_features = ['nom_' + str(i) for i in range(10)]

In [ ]:
from sklearn.preprocessing import OneHotEncoder

onehot_encoder = OneHotEncoder() # OneHotEncoder 객체 생성
# 원-핫 인코딩 적용
encoded_nom_matrix = onehot_encoder.fit_transform(all_data[nom_features])

# 기존 명목형 피처 삭제
all_data = all_data.drop(nom_features, axis=1)

### 날짜형 피처

In [ ]:
date_features  = ['day', 'month'] # 날짜 피처

# 원-핫 인코딩 적용
encoded_date_matrix = onehot_encoder.fit_transform(all_data[date_features])

# 기존 날짜 피처 삭제
all_data = all_data.drop(date_features, axis=1)

### 피처 스케일링~

In [ ]:
from sklearn.preprocessing import MinMaxScaler

ord_features = ['ord_' + str(i) for i in range(6)] # 순서형 피처
# min-max 정규화
all_data[ord_features] = MinMaxScaler().fit_transform(all_data[ord_features])

### 인코딩, 스케일링된 피처 합치기

In [ ]:
from scipy import sparse

# 인코딩 및 스케일링된 피처 합치기
all_data_sprs = sparse.hstack([sparse.csr_matrix(all_data),
                               encoded_nom_matrix,
                               encoded_date_matrix],
                              format='csr')

all_data_sprs

### Split

In [ ]:
num_train = len(train) # 훈련 데이터 개수

# 훈련 데이터와 테스트 데이터 나누기
X_train = all_data_sprs[:num_train] # 0 ~ num_train - 1행
X_test = all_data_sprs[num_train:] # num_train ~ 마지막 행

y = train['target']

### optimization of hyperparameters

In [ ]:
# Grid Search

# Check time - 7mins
# %%time 

from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

# 로지스틱 회귀 모델 생성
logistic_model = LogisticRegression()

# 하이퍼파라미터 값 목록
lr_params = {'C':[0.1, 0.125, 0.2], 'max_iter':[800, 900, 1000], 
             'solver':['liblinear'], 'random_state':[42]}

# 그리드서치 객체 생성
gridsearch_logistic_model = GridSearchCV(estimator=logistic_model,
                                         param_grid=lr_params,
                                         scoring='roc_auc', # 평가지표
                                         cv=5)
# 그리드서치 수행
gridsearch_logistic_model.fit(X_train, y)

print('최적 하이퍼파라미터:', gridsearch_logistic_model.best_params_)

In [ ]:
# 타깃값 1일 확률 예측
y_preds = gridsearch_logistic_model.best_estimator_.predict_proba(X_test)[:,1]

# 제출 파일 생성
submission['target'] = y_preds
submission.to_csv('submission.csv')